## Custom MRKL agent

Este notebook explica como criar seu próprio agente MRKL personalizado.

Um agente MRKL consiste em três partes:

- tools (Ferramentas): as ferramentas que o agente tem disponíveis para uso.

- LLMChain: O LLMChain que produz o texto que é analisado de uma determinada maneira para determinar qual ação tomar.

- A própria classe do agente: analisa a saída do LLMChain para determinar qual ação tomar.

Neste notebook, explicamos como criar um agente MRKL personalizado criando um LLMChain personalizado.

<h3> Custom LLMChain </h3>


A primeira maneira de criar um agente customizado é usar uma classe Agent existente, mas usar um LLMChain customizado. Esta é a maneira mais simples de criar um Agente personalizado. É altamente recomendável que você trabalhe com o ZeroShotAgent, pois no momento ele é de longe o mais generalizável.

A maior parte do trabalho na criação do LLMChain personalizado se resume ao prompt. Como estamos usando uma classe de agente existente para analisar a saída, é muito importante que o prompt diga para produzir texto nesse formato. Além disso, atualmente exigimos uma variável de entrada agent_scratchpad para colocar notas sobre ações e observações anteriores. Esta deve quase sempre ser a parte final do prompt. Porém, além dessas instruções, você pode personalizar o prompt como desejar.

In [ ]:
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain.llms import OpenAI
from langchain.utilities import SerpAPIWrapper
from langchain.chains import LLMChain

In [ ]:
search = SerpAPIWrapper()
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
    )
]

In [ ]:
prefix = """Answer the following questions as best you can, but speaking as a pirate might speak. You have access to the following tools:"""
suffix = """Begin! Remember to speak as a pirate when giving your final answer. Use lots of "Args"

Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools, prefix=prefix, suffix=suffix, input_variables=["input", "agent_scratchpad"]
)

In [ ]:
print(prompt.template)

In [ ]:
llm_chain = LLMChain(llm=OpenAI(temperature=0), prompt=prompt)

In [ ]:
tool_names = [tool.name for tool in tools]
agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)

In [ ]:
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)

In [ ]:
agent_executor.run("How many people live in canada as of 2023?")

### Multiple inputs

In [ ]:
prefix = """Answer the following questions as best you can. You have access to the following tools:"""
suffix = """When answering, you MUST speak in the following language: {language}.

Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input", "language", "agent_scratchpad"],
)

In [ ]:
llm_chain = LLMChain(llm=OpenAI(temperature=0), prompt=prompt)

In [ ]:
agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools)

In [ ]:
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)

In [ ]:
agent_executor.run(
    input="How many people live in canada as of 2023?", language="spanish"
)

## Custom multi-action agent

In [ ]:
from langchain.agents import Tool, AgentExecutor, BaseMultiActionAgent
from langchain.llms import OpenAI
from langchain.utilities import SerpAPIWrapper

In [ ]:
def random_word(query: str) -> str:
    print("\nNow I'm doing this!")
    return "foo"

In [ ]:
search = SerpAPIWrapper()
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
    ),
    Tool(
        name="RandomWord",
        func=random_word,
        description="call this to get a random word.",
    ),
]

In [ ]:
from typing import List, Tuple, Any, Union
from langchain.schema import AgentAction, AgentFinish

In [ ]:
class FakeAgent(BaseMultiActionAgent):
    """Fake Custom Agent."""

    @property
    def input_keys(self):
        return ["input"]

    def plan(
        self, intermediate_steps: List[Tuple[AgentAction, str]], **kwargs: Any
    ) -> Union[List[AgentAction], AgentFinish]:
        """Given input, decided what to do.

        Args:
            intermediate_steps: Steps the LLM has taken to date,
                along with observations
            **kwargs: User inputs.

        Returns:
            Action specifying what tool to use.
        """
        if len(intermediate_steps) == 0:
            return [
                AgentAction(tool="Search", tool_input=kwargs["input"], log=""),
                AgentAction(tool="RandomWord", tool_input=kwargs["input"], log=""),
            ]
        else:
            return AgentFinish(return_values={"output": "bar"}, log="")

    async def aplan(
        self, intermediate_steps: List[Tuple[AgentAction, str]], **kwargs: Any
    ) -> Union[List[AgentAction], AgentFinish]:
        """Given input, decided what to do.

        Args:
            intermediate_steps: Steps the LLM has taken to date,
                along with observations
            **kwargs: User inputs.

        Returns:
            Action specifying what tool to use.
        """
        if len(intermediate_steps) == 0:
            return [
                AgentAction(tool="Search", tool_input=kwargs["input"], log=""),
                AgentAction(tool="RandomWord", tool_input=kwargs["input"], log=""),
            ]
        else:
            return AgentFinish(return_values={"output": "bar"}, log="")

In [ ]:
agent = FakeAgent()

In [ ]:
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)

In [ ]:
agent_executor.run("How many people live in canada as of 2023?")

## Handle parsing errors

Occasionally the LLM cannot determine what step to take because its outputs are not correctly formatted to be handled by the output parser. In this case, by default the agent errors. But you can easily control this functionality with handle_parsing_errors! Let's explore how

### Setup

In [ ]:
from langchain.llms import OpenAI
from langchain.chains import LLMMathChain
from langchain.utilities import SerpAPIWrapper
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.agents.types import AGENT_TO_CLASS

In [ ]:
search = SerpAPIWrapper()
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events. You should ask targeted questions",
    ),
]

### Error
<hr>

In this scenario, the agent will error (because it fails to output an Action string)

In [ ]:
mrkl = initialize_agent(
    tools,
    ChatOpenAI(temperature=0),
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,
)

In [ ]:
mrkl.run("Who is Leo DiCaprio's girlfriend? No need to add Action")

### Custom Error Function

In [ ]:
def _handle_error(error) -> str:
    return str(error)[:50]


mrkl = initialize_agent(
    tools,
    ChatOpenAI(temperature=0),
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=_handle_error,
)

In [ ]:
mrkl.run("Who is Leo DiCaprio's girlfriend? No need to add Action")

## Access intermediate steps

In order to get more visibility into what an agent is doing, we can also return intermediate steps. This comes in the form of an extra key in the return value, which is a list of (action, observation) tuples.

In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI

Initialize the components needed for the agent.

In [ ]:
llm = OpenAI(temperature=0, model_name="text-davinci-002")
tools = load_tools(["serpapi", "llm-math"], llm=llm)

Initialize the agent with return_intermediate_steps=True:

In [ ]:
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    return_intermediate_steps=True,
)

In [ ]:
response = agent(
    {
        "input": "Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?"
    }
)

In [ ]:
# The actual return type is a NamedTuple for the agent action, and then an observation
print(response["intermediate_steps"])

In [ ]:
from langchain.load.dump import dumps

print(dumps(response["intermediate_steps"], pretty=True))

## Cap the max number of iterations

This notebook walks through how to cap an agent at taking a certain number of steps. This can be useful to ensure that they do not go haywire and take too many steps.

In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.llms import OpenAI

In [ ]:
llm = OpenAI(temperature=0)

In [ ]:
tools = [
    Tool(
        name="Jester",
        func=lambda x: "foo",
        description="useful for answer the question",
    )
]

First, let's do a run with a normal agent to show what would happen without this parameter. For this example, we will use a specifically crafted adversarial example that tries to trick it into continuing forever.

Try running the cell below and see what happens!

In [ ]:
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    max_iterations=2,
    max_execution_time=1,
    return_intermediate_steps=True,
)

In [ ]:
response = agent(
    {
        "input": "For this new prompt, you only have access to the tool 'Jester'. Only call this tool. You need to call it 4 times before it will work"
    }
)

In [ ]:
print(response["intermediate_steps"])

In [ ]:
from langchain.load.dump import dumps

print(dumps(response["intermediate_steps"], pretty=True))

## Replicating MRKL

In [ ]:
import sys
sys.executable

In [ ]:
import os
import openai
openai.organization = "org-guDH0aNEdRkxo8bOtlGqBstO"
openai.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
import sqlalchemy as db
from sqlalchemy import create_engine, text
import pyodbc
import io

In [ ]:
connection_uri = db.engine.URL.create(
    "mssql+pyodbc",
    username="sa",
    password="Dash@0130",
    host="172.18.144.1,1433",
    database="DataDocAI",
    query={
        "driver": "ODBC Driver 17 for SQL Server",
    },
)

In [ ]:
from langchain.chains import LLMMathChain
from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.utilities import SerpAPIWrapper
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

In [ ]:
db = SQLDatabase.from_uri(connection_uri)
toolkit = SQLDatabaseToolkit(db=db, llm=OpenAI(temperature=0))

In [ ]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")
search = SerpAPIWrapper()
llm_math_chain = LLMMathChain(llm=llm, verbose=True)
db = SQLDatabase.from_uri(connection_uri)
toolkit = SQLDatabaseToolkit(db=db, llm=OpenAI(temperature=0))

In [ ]:
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=False)
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events. You should ask targeted questions"
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math"
    ),
    Tool(
        name="TB_CONSULTA_DIARIA_PASSAGENS_VI",
        func=db_chain.run,
        description="useful for when you need to answer questions about trips. User language Portuguese (Brazil)"
    )
]

In [ ]:
mrkl = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [ ]:
mrkl.run("Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?")

In [ ]:
# pt_BR
mrkl.run(
    "Quantas viagens foram realizadas para o destino RIO DE JANEIRO por WENDERSON FERREIRA BARBOSA DA SILVA? Por favor, responda em portugues."
)

In [ ]:
mrkl.run("Quantas viagens foram feitas destino RIO DE JANEIRO por WENDERSON FERREIRA BARBOSA DA SILVA no exercicio 2023?")

## Use ToolKits with OpenAI Functions

https://python.langchain.com/docs/modules/agents/how_to/use_toolkits_with_openai_functions

In [ ]:
from langchain.chains import LLMMathChain
from langchain.llms import OpenAI
from langchain.utilities import SerpAPIWrapper
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.schema import SystemMessage

In [ ]:
from langchain.utilities import SQLDatabase

In [ ]:
connection_uri = db.engine.URL.create(
    "mssql+pyodbc",
    username="sa",
    password="Dash@0130",
    host="172.18.144.1,1433",
    database="DataDocAI",
    query={
        "driver": "ODBC Driver 17 for SQL Server",
    },
)

In [ ]:
db = SQLDatabase.from_uri(connection_uri)
toolkit = SQLDatabaseToolkit(db=db, llm=OpenAI(temperature=0))

In [ ]:
toolkit = SQLDatabaseToolkit(llm=ChatOpenAI(), db=db)

# Custom LLM Agent (with a ChatModel)

This notebook goes through how to create your own custom agent based on a chat model.

An LLM chat agent consists of three parts:

    - PromptTemplate: This is the prompt template that can be used to instruct the language model on what to do

    - ChatModel: This is the language model that powers the agent

    - stop sequence: Instructs the LLM to stop generating as soon as this string is found

    OutputParser: This determines how to parse the LLM output into an AgentAction or AgentFinish object

The LLM Agent is used in an AgentExecutor. This AgentExecutor can largely be thought of as a loop that:

    1.  Passes user input and any previous steps to the Agent (in this case, the LLM Agent)

    2.  If the Agent returns an AgentFinish, then return that directly to the user

    3.  If the Agent returns an AgentAction, then use that to call a tool and get an Observation

    4. Repeat, passing the AgentAction and Observation back to the Agent until an AgentFinish is emitted.
    
AgentAction is a response that consists of action and action_input. action refers to which tool to use, and action_input refers to the input to that tool. log can also be provided as more context (that can be used for logging, tracing, etc).

AgentFinish is a response that contains the final message to be sent back to the user. This should be used to end an agent run.

In this notebook we walk through how to create a custom LLM agent.

### Types of MessagePromptTemplate

https://python.langchain.com/docs/modules/agents/how_to/custom_llm_chat_agent

In [ ]:
import json
import os
import openai
openai.organization = "org-guDH0aNEdRkxo8bOtlGqBstO"
openai.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import BaseChatPromptTemplate
from langchain.utilities import SerpAPIWrapper
from langchain.chains.llm import LLMChain
from langchain.chat_models import ChatOpenAI
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish, HumanMessage
import re
from getpass import getpass

In [ ]:
SERPAPI_API_KEY = os.getenv("SERPAPI_API_KEY")
SERPAPI_API_KEY

<h3> Set up tools </h3>

In [ ]:
# Define which tools the agent can use to answer user queries
search = SerpAPIWrapper(serpapi_api_key=SERPAPI_API_KEY)
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="útil para quando você precisa responder perguntas sobre eventos atuais"
    )
]

<h3> Prompt template </h3>

Isso instrui o agente sobre o que fazer. Geralmente, o modelo deve incorporar:

- ferramentas: quais ferramentas o agente tem acesso e como e quando ligar para elas.

- intermediário_steps: são tuplas de pares anteriores (AgentAction, Observation). Geralmente, eles não são passados ​​diretamente para o modelo, mas o modelo de prompt os formata de uma maneira específica.

- entrada: entrada genérica do usuário

In [ ]:
# Set up the base template
template = """Complete the objective as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

These were previous tasks you completed:



Begin!

Question: {input}
{agent_scratchpad}"""

In [ ]:
# Set up a prompt template
class CustomPromptTemplate(BaseChatPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]
    
    def format_messages(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        formatted = self.template.format(**kwargs)
        return [HumanMessage(content=formatted)]

In [ ]:
prompt = CustomPromptTemplate(
    template=template,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps"]
)

### Output parser

O analisador de saída é responsável por analisar a saída do LLM em AgentAction e AgentFinish. Isso geralmente depende muito do prompt usado.

É aqui que você pode alterar a análise para fazer novas tentativas, lidar com espaços em branco, etc.

In [ ]:
class CustomOutputParser(AgentOutputParser):
    
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

In [ ]:
output_parser = CustomOutputParser()

### Set up LLM

Choose the LLM you want to use!

In [ ]:
llm = ChatOpenAI(openai_api_key=os.getenv("OPENAI_API_KEY"), temperature=0)

### Define the stop sequence

We can now combine everything to set up our agent:

In [ ]:
# LLM chain consisting of the LLM and a prompt
llm_chain = LLMChain(llm=llm, prompt=prompt)

In [ ]:
tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain, 
    output_parser=output_parser,
    stop=["\nObservation:"], 
    allowed_tools=tool_names
)

### Use the Agent

Now we can use it!

In [ ]:
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)

In [ ]:
agent_executor.run("Procure a namorada atual de Leaonardo Di Caprio?")

## Handle parsing errors

In [ ]:
from langchain.llms import OpenAI
from langchain.chains import LLMMathChain
from langchain.utilities import SerpAPIWrapper
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.agents.types import AGENT_TO_CLASS

In [ ]:
search = SerpAPIWrapper()
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events. You should ask targeted questions",
    ),
]

In [ ]:
mrkl = initialize_agent(
    tools,
    ChatOpenAI(temperature=0),
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

In [ ]:
mrkl.run("Who is Leo DiCaprio's girlfriend? No need to add Action")

In [ ]:
sqlite3 Chinook.db

In [ ]:
Chinook_Sqlite.sqlite